# Clustering python notebook

This notebook is used as the basic notebook to run all subsequent clustering scripts across the 
datasets. 

Clustering is used to identify groups within data that have similar underlying characteristics, while those in other clusters are seen as different. The idea is to make sure that the data points in one cluster are as similar as possible while being different compared to other clusters. For our purpose this means that we want to find MSOA that have similar transport characteristics such that they create a subgroup of MSOAs.

This is an unsupervised machine learning methodology whereby we don't have already labeled data and thus the purpose of this is to create the labels. Hence, we don't have anything to compare the outcomes to. 

The first step is to clean the data to make sure that it is in the right format to run the algorithms below. The data that is fed in here has already gone through the transformation and standardisation process and so it is just making sure that it is in the correct format. 

In [1]:
import os 
os.getcwd()

'C:\\Users\\cex\\Documents\\Smart Cities and Urban Analytics\\Term 2\\SDC\\Assessment'

In [ ]:
# importing libraries required
import pandas as pd
import plotly.express as px
import sklearn.preprocessing as preprocessing
from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import random
from sklearn.cluster import AgglomerativeClustering

# run this command too - just to allow more data to be displayed than default
pd.set_option('display.max_rows', 200)
# this one ensures graphs properly display in the notebook
%matplotlib inline

In [ ]:
#define the filepath
file_path = ""

In [ ]:
#put the data into a dataframe
data = pd.read_csv()
#check that is has been read correctly
data

In [ ]:
#if we want to merge by origin
msoa = data.groupby("Area of residence").sum()

In [ ]:
#to automatically scale the dataframe

#this takes arguments: X ,axis = 0, with_mean = True, with_std = True, copy = True
#X is the data to centre and scale
#axs: int (0 by default) - axis used to compute means and Std along, if 0  independentky
#standardise each feature, other (if 1) standardize each sample
#with_mean: boolean, True by default, if true centre the data before scaling
#with_std: boolean, True by default, if True scale the data to unit variance
#copy: boolean, optional, default True, set to False to perform inplace row normalisation and avoid a copy

#NaNs are treated as missing values: disregarded

scaled = preprocessing.scale(transport_percentage) # add your dataframe name here

#this creates a numpy array
scaled

In [ ]:
#to change the numpy array back into a dataframe
scaled_df = pd.DataFrame({'Private_transport': scaled[:, 0], 'Public _transport': scaled[:, 1], 'People_power':scaled[:, 2]})
print(scaled_df)

## DBScan

DBScan creates a clustering of points based on local proximity. This means that it assigns points to clusters only when a certain number of points lie within a given distance threshold, often given by euclidean distance. In effect this method seperates clusters on the basis of density in terms of the number of points inthe surrounding areas.

This works by starting at a random point in the dataset. If the minimum number of points as stipulated in the `min_samples` variable surroundings the point within the threshold distance then a cluster is formed. If the point does not satisfy this criteria then it is labelled as noise. This process continues throughout all points, merging clusters where they already exist such that we get the outcome of the number of clusters.

The benefit of this is that it can identify outliers well, as those that are not within a given density of points, and thus does not have to cluster every single point. 

The issue is that this does not deal well with uniform data points or clusters with different densities, you need prior knowledge of the data to specify the distance measure and the minimum number of points, and does not do well when data has a high degree of density.

In [ ]:
# create DBSCAN cluster object

#eps is the maximum distance between two samples for one to be considered as in the neighbourhood of other other
#min samples is the number of samples within that distance to make that point a core point, this includes the point itself
#this also takes several other arguments e.g.
#metric= 'euclidean', metric_params=None, algorith='auto', leaf_szie=30, p = None, n_jobs=None
dbscan = DBSCAN(eps=0.1, min_samples=5) 

#algorith to be used by the NearestNeighbors module to compute pointwise distance and find neighest neighbours - see NN module documentation
# run the .fit() function on the scaled dataset
dbscan.fit(scaled) 

In [ ]:
#Get the labels from the clusters
dbscan_labels = dbscan.labels_

In [ ]:
#check the silhouette score
metrics.silhouette_score(scaled, dbscan_labels)

In [ ]:
#assign the labels to the original dataframe
transport_percentage=transport_percentage.assign(label = dbscan_labels)

#and check the calue counts
transport_percentage.label.value_counts()

In [ ]:
#check the number of clusters
n_clusters_ = len(set(dbscan_labels)) - (1 if -1 in dbscan_labels else 0)
#check the number of points outside the cluster
n_noise_ = list(dbscan_labels).count(-1)

#print the number of clusters
print(n_clusters_)
#print the number of points outside the cluster
print(n_noise_)

In [ ]:
#compute the silhoutte score
print("The Silhouette score is: %0.2f" % metrics.silhouette_score(scaled, dbscan_labels))

The silhouette scale can be used to measure the performance of dbscan. This is calculated using the mean intra-cluster distance between points and the mean nearest-cluster distance. This ranges from -1 to 1 with 1 being the best score

# Kmeans clustering

The Kmeans clustering algorithm works by breaking down the dataset into groups based on proximity of points. This is based intially on a predefined number of centroids randomly assigned within space. This then assigns individual points to their nearest centroid. Once this is complete, the centroid of the space is recalculted and repositioned such that it lies within the centre of the given points. Points are then reassigned based on their distance to the centroid and the process occurs iteratively until there is no more reassignment.

The issue with this methodology is that it requires a knowledge of the number of clusters (although this can be solved through the eblow methods), this is sensitive to where the centroids are initially placed (although this can be solved through repeat measurements of silhouette analysis), this is sensitive to outliers and is incapable of handling clusters of non-convex shape.

In [ ]:
#set the number of clusters to explore
k_cluster = 5
#set the random seed
random_seed = 1
#random state: int, default = None
#determines random number generation for ecntroid intialisation, use an int to make the randomness deterministic

In [ ]:
#get the method for kmeans
kmeans_method = KMeans(n_clusters=k_cluster,random_state=random_seed)
#apply the fit to the scaled dataset
kmeans_method.fit(scaled)

In [ ]:
#assign the labels to original databse
transport_percenatege_kmeans = transport_percentage.assign(label = kmeans_method.labels_)

### Evaluation

This method can be evaluated using:

1) Elbow Method
    - This gives us an idea of what a good number of clusters would be based on the sum of squared (SSE) distance between data points and their assigned centroids
    - The number is picked based on where the SSE begins to flatten out and forms an elbow
    - This can sometimes not show an elbow and therefore we can support this with Silhouette analysis
2) Silhouette Analysis
    - This can be used to determine how seperated clusters as explained above
    - The idea is to make sure that the coefficients are as large as possible

In [ ]:
#elbow plot over multiple k's

#calculate SSE for a range of number of cluster

list_SSE = []
min_k = 1
max_k = 10
range_k = range(min_k, max_k+1)
for i in range_k:
    km = KMeans(
        n_clusters=i, init='random',
        n_init=10, max_iter=300,
        tol=1e-04, random_state=0
    )
    km.fit(scaled)
    # inertia is a concept in physics. Roughly it means SSE of clustering.
    list_SSE.append(km.inertia_)

# plot
plt.plot(range_k, list_SSE, marker='o')
plt.xlabel('Number of clusters')
plt.ylabel('SSE')
plt.show()

In [ ]:
#silhouette score over multiple k's

#empty array to hold the silhouette scores
silhouette = []

#iterate over the number of clusters from 2 to 10
for i in range(2, 21):
    #create an empty array to store the average
    average = []
    #repeat the kmeans clustering 10 times
    for x in range(1, 10):
        #setting the number of clusters as i
        k_cluster = i
        #create a random integer for the random seed
        random_seed = random.randint(1,101)
        #run the kmeans analysis
        kmeans_method = KMeans(n_clusters=k_cluster,random_state=random_seed)
        #fit it to the scaled dataset
        kmeans_method.fit(scaled)
        #get the labels
        labels = kmeans_method.labels_
        #get the silhouette score
        a = metrics.silhouette_score(scaled, labels)
        #append it to the average list
        average.append(a)
    #get the silhouette score and append it to the silhouette 
    silhouette.append(sum(average)/len(average))
    
#plot the silhouette score
plt.plot(silhouette)

## Agglomerative clustering

In [ ]:
#set the number of clusters expected
n_clusters = 4

#get the hierarchy method
hierarchy = AgglomerativeClustering(n_clusters = n_clusters)

#fit it to the data
hierarchy.fit(scaled)

In [ ]:
#assign labels to the original data
transport_percentage_hierarchy = transport_percentage.assign(label = hierarchy.labels_)

In [ ]:
hierarchy_labels = hierarchy.labels_

#check the silhouette score
metrics.silhouette_score(scaled, hierarchy_labels)